# 基于模型的协同过滤算法

读取数据

In [1]:
import math
import numpy as np

def load_movielens(path='./ml-100k'):
    
    # get movie titles
    movies = {}
    for line in open(path + '/u.item', encoding='latin-1'):
        id, title = line.split('|')[0:2]
        movies[id] = title
    # load data
    prefs = {}
    for line in open(path + '/u.data', encoding='latin-1'):
        user, movieid, rating, ts = line.split('\t')
        prefs.setdefault(user, {})
        prefs[user][movies[movieid]] = float(rating)
    return prefs

prefs = load_movielens()['7']
print(prefs.keys())

dict_keys(['Crumb (1994)', 'Vertigo (1958)', "Jackie Chan's First Strike (1996)", 'Adventures of Priscilla, Queen of the Desert, The (1994)', 'Return of the Pink Panther, The (1974)', 'Duck Soup (1933)', 'Bringing Up Baby (1938)', 'East of Eden (1955)', 'High Noon (1952)', 'Quiet Man, The (1952)', 'Miracle on 34th Street (1994)', 'Shining, The (1980)', 'Misérables, Les (1995)', 'Waterworld (1995)', 'Dragonheart (1996)', 'Angels in the Outfield (1994)', 'Thin Blue Line, The (1988)', 'Somewhere in Time (1980)', 'Delicatessen (1991)', 'Grease (1978)', 'Cat People (1982)', 'Crucible, The (1996)', 'Brady Bunch Movie, The (1995)', 'Hudsucker Proxy, The (1994)', 'Cliffhanger (1993)', 'Star Trek: The Wrath of Khan (1982)', 'Willy Wonka and the Chocolate Factory (1971)', 'Englishman Who Went Up a Hill, But Came Down a Mountain, The (1995)', 'Kalifornia (1993)', 'Diabolique (1996)', 'Killing Fields, The (1984)', 'The Innocent (1994)', 'Seventh Seal, The (Sjunde inseglet, Det) (1957)', 'Piano, Th

建立模型：基于矩阵分解的模型推荐算法

In [ ]:
def sgd(data_matrix, k, alpha, lam, max_cycles):
    """使用梯度下降法进行矩阵分解。

    Args:
    - data_matrix: mat, 用户物品矩阵
    - k: int, 分解矩阵的参数
    - alpha: float, 学习率
    - lam: float, 正则化参数
    - max_cycles: int, 最大迭代次数

    Returns:
    p,q: mat, 分解后的矩阵
    """
    m, n = np.shape(data_matrix)
    # initiate p & q
    p = np.mat(np.random.random((m, k)))
    q = np.mat(np.random.random((k, n)))

    # start training
    for step in range(max_cycles):
        for i in range(m):
            for j in range(n):
                if data_matrix[i, j] > 0:
                    error = data_matrix[i, j]
                    for r in range(k):
                        error = error - p[i, r] * q[r, j]
                    for r in range(k):
                        p[i, r] = p[i, r] + alpha * (2 * error * q[r, j] - lam * p[i, r])
                        q[r, j] = q[r, j] + alpha * (2 * error * p[i, r] - lam * q[r, j])

        loss = 0.0
        for i in range(m):
            for j in range(n):
                if data_matrix[i, j] > 0:
                    error = 0.0
                    for r in range(k):
                        error = error + p[i, r] * q[r, j]
                    # calculate loss function
                    loss = (data_matrix[i, j] - error) * (data_matrix[i, j] - error)
                    for r in range(k):
                        loss = loss + lam * (p[i, r] * p[i, r] + q[r, j] * q[r, j]) / 2

        if loss < 0.001:
            break
        if step % 1000 == 0:
            print("\titer: %d, loss: %f" % (step, loss))
    return p, q

在ml-100k上进行测试